In [ ]:
#!/usr/bin/env python3
import os, json, glob

SITE_ROOT   = r"C:\Users\user\Desktop\FatemehUni\master\project\understanding_visual_reasoning\logit_lens\vis1"

# SITE_ROOT   = os.path.dirname(os.path.abspath(__file__))
REPORT_DIR  = os.path.join(SITE_ROOT, "reports")  # where your logitlens_once_*.json files are
OUT_JSON    = os.path.join(SITE_ROOT, "index.json")

def to_rel_image_from_image_path(abs_or_rel_path: str) -> str:
    """
    From JSON["image_path"], extract a path starting at the 'data/' segment:
      '/home/.../data/images_row/trial-20_0.png' -> 'data/images_row/trial-20_0.png'
    If no 'data' segment is found, fallback to the basename.
    """
    p = abs_or_rel_path.replace("\\", "/")
    parts = p.split("/")
    if "data" in parts:
        i = parts.index("data")
        return "/".join(parts[i:])
    return os.path.basename(p)

def main():
    items = []
    pattern = os.path.join(REPORT_DIR, "logitlens_once_*.json")
    for jpath in sorted(glob.glob(pattern)):
        try:
            with open(jpath, "r", encoding="utf-8") as f:
                payload = json.load(f)
        except Exception as e:
            print(f"[skip] {jpath} (read error: {e})")
            continue

        # image (relative from 'data/')
        img_rel = to_rel_image_from_image_path(payload.get("image_path", ""))

        # name label
        name = os.path.splitext(os.path.basename(img_rel))[0] if img_rel else os.path.splitext(os.path.basename(jpath))[0]

        # prompt (optional)
        prompt = payload.get("prompt") or payload.get("QUESTION") or ""

        # store json path relative to site root (use forward slashes)
        json_rel = os.path.relpath(jpath, SITE_ROOT).replace("\\", "/")

        items.append({
            "name": name,
            "json": json_rel,
            "image": img_rel,   # e.g. data/...
        })

    with open(OUT_JSON, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

    print(f"Wrote {OUT_JSON} with {len(items)} items")

if __name__ == "__main__":
    main()


Wrote C:\Users\user\Desktop\FatemehUni\master\project\understanding_visual_reasoning\logit_lens\vis1\index.json with 100 items
